In [9]:
import sys
print(f"Python: {sys.version}")
print(f"Executable: {sys.executable}")
# --- Load env vars from repo root --- 
from pathlib import Path
from dotenv import load_dotenv
# --- Load .env from repo root --- 
env_path = Path().resolve().parent.parent / ".env"
load_dotenv(env_path)

print(f"Environment variables loaded from: {env_path}")

Python: 3.12.10 (main, Apr  9 2025, 04:06:22) [MSC v.1943 64 bit (AMD64)]
Executable: c:\Users\PCS\Desktop\Projects\servless-rag\.venv\Scripts\python.exe
Environment variables loaded from: C:\Users\PCS\Desktop\Projects\.env


In [10]:

# --- Create the Client --- 
from google import genai 
from google.genai import types  
import os 
# --- Get API Key --- 
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

if not GEMINI_API_KEY:
    raise ValueError("GEMINI_API_KEY is not set")
    
client = genai.Client(api_key=GEMINI_API_KEY)

print("Client created successfully")





Client created successfully


In [11]:
import time 
# --- Create store with descriptive name --- 
store = client.file_search_stores.create(
    # --- Add name of the store here ---- 
    config={
        "display_name": "test-store-servless-rag-001",
    }
)
print(f"Store created successfully: {store.name}")
print(f"Store ID: {store.name.split('/')[-1]}")

Store created successfully: fileSearchStores/teststoreservlessrag001-i2bikwowxixi
Store ID: teststoreservlessrag001-i2bikwowxixi


In [12]:
# --- Create a Sample Text File --- 
sample_content = """
Robert Graves (1895-1985) was an English poet, novelist, and classicist. 
He wrote "I, Claudius" (1934), a historical novel about the Roman Emperor. 
Graves was also a renowned translator of ancient texts.
"""

with open("sample.txt", "w") as f:
    f.write(sample_content)

# --- let's Upload directly to the store --- 
# --- This will (chunks , embeds , and indexes automatically ) --- 
operation = client.file_search_stores.upload_to_file_search_store(
    file='sample.txt',
    file_search_store_name=store.name,
    config={'display_name': 'Robert Graves Bio'}
)

print(f"⏳ Import operation started: {operation.name}")

# --- Wait for completion --- 
while not operation.done:
    time.sleep(5)
    operation = client.operations.get(operation)
    print(f"Status : {operation.metadata}")

print("File imported successfully ")

⏳ Import operation started: fileSearchStores/teststoreservlessrag001-i2bikwowxixi/upload/operations/robert-graves-bio-enq48yvipwmk
Status : None
File imported successfully 


In [ ]:
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="What was Robert Graves famous for?",
    config=types.GenerateContentConfig(
        tools=[
            types.Tool(
                file_search=types.FileSearch(
                    file_search_store_names=[store.name]
                )
            )
        ]
    )
)

print("🤖 AI Response:")
print(response.text)

# ---  Show grounding metadata ---
if response.candidates and response.candidates[0].grounding_metadata:
    print("\n📚 Grounding Metadata (Citations):")
    print(response.candidates[0].grounding_metadata)

🤖 AI Response:
Robert Graves (1895-1985) was an English poet, novelist, and classicist who gained fame for several contributions. He is widely recognized for his historical novel "I, Claudius," published in 1934, which tells the story of the Roman Emperor. Additionally, Graves was a renowned translator of ancient texts. He was also known for his extensive work as a poet.

📚 Grounding Metadata (Citations):
google_maps_widget_context_token=None grounding_chunks=[GroundingChunk(
  retrieved_context=GroundingChunkRetrievedContext(
    text="""
Robert Graves (1895-1985) was an English poet, novelist, and classicist. 
He wrote "I, Claudius" (1934), a historical novel about the Roman Emperor. 
Graves was also a renowned translator of ancient texts.
""",
    title='Robert Graves Bio'
  )
)] grounding_supports=[GroundingSupport(
  grounding_chunk_indices=[
    0,
  ],
  segment=Segment(
    end_index=241,
    start_index=115,
    text='He is widely recognized for his historical novel "I, Claudi

In [16]:
amir_secreate="Amir's biggest secret is that he loves Warp more than Cursor."
with open("amir.txt","w") as f:
    f.write(amir_secreate)
# --- Upload directly to store --- 
# --- so here (the Chunks , embeds , and indexes are automcatically) --- 
operation = client.file_search_stores.upload_to_file_search_store(
    file="amir.txt",
    file_search_store_name=store.name,
    config={
        'display_name': 'Amir'
    }
)
print(f"Import Operation started:{operation.name}")
# --- Wait for Completion --- 
while not operation.done:
    time.sleep(5)
    operation = client.operations.get(operation)
    print(f"Status:{operation.metadata}")

print("File Imported Successfully ! ")


Import Operation started:fileSearchStores/teststoreservlessrag001-i2bikwowxixi/upload/operations/amir-yjiljfnaf4w3
Status:None
File Imported Successfully ! 


In [18]:
# --- Quick Test ---
response = client.models.generate_content(
    model="gemini-flash-latest",
    contents="What is Amir Biggest Secrete Please",
    config=types.GenerateContentConfig(
        tools=[
            types.Tool(
                file_search=types.FileSearch(
                    file_search_store_names=[store.name] 
                )
            )
        ]
    )
)
print("🤖 AI Response:")
print(response.text)

🤖 AI Response:
Based on the information I have, Amir's biggest secret is that **he loves Warp more than Cursor** .
